In [1]:
import torch

In [2]:
import numpy as np
import csv
import random
import math
import matplotlib

import matplotlib.pyplot as plt

%matplotlib inline

#import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import copy
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as TF
import torchvision.datasets as datasets
import torch.optim as optim
from torch import utils
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
use_cuda = torch.cuda.is_available()
torch.cuda.set_device(1)
CUDA_LAUNCH_BLOCKING=1
#torch.autograd.set_detect_anomaly(True)
print (use_cuda)

torch.backends.cudnn.enabled
import os


import time

True


In [3]:
import argparse
import os
import shutil
import sys
import time

In [4]:
import numpy as np
import torch
import torch.nn as nn
from sklearn import metrics
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [5]:
from cgcnn.data import CIFData
from cgcnn.data import collate_pool
from cgcnn.model import CrystalGraphConvNet

In [6]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.db import connect


from clease.tools import update_db
from clease import Concentration
from clease import CEBulk
from clease import Evaluate
from clease import NewStructures
from clease.calculator import Clease
from clease.calculator import attach_calculator

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt
import numpy as np


conc = Concentration(basis_elements=[['Ag', 'Pd']])
settings = CEBulk(crystalstructure='fcc',
                   a=4.09,
                   size=[3,3,3],
                   concentration=conc,
                   db_name="agpd_dft_ref_final_submission.db",
                   max_cluster_size=4,
                   max_cluster_dia=[8.0, 6.5, 5.5])

atoms = settings.atoms.copy()
k_b = 0.00008617

def get_concentrations(lattices):
    lattices = ((lattices+1)/2).view(lattices.shape[0],-1)
    Ag_conc = torch.sum(lattices,dim=1)/(27)
    return Ag_conc

In [7]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ase.io.trajectory import TrajectoryWriter
from ase.io import read
from ase.io import iread
from ase.io import write



lst = []
for i in range(1):
    name = 'batch_template_1/structure_num' + str(i) + '.cif'
    for j in range(len(atoms.numbers)):
        rd = np.random.rand()
        if rd > 0.5:
            atoms.numbers[j] = 46
        else:
            atoms.numbers[j] = 47
    
    print(atoms)
    write(name,atoms)
    
    curr_entry = []
    curr_entry.append('structure_num' + str(i))
    curr_entry.append(0)
    lst.append(curr_entry)
    
df = pd.DataFrame(lst)
file = 'batch_template_1/id_prop.csv'
df.to_csv(file, header=None, index=False)

Atoms(symbols='Pd2AgPd5AgPdAg2Pd5AgPd2Ag2PdAg4', pbc=True, cell=[[0.0, 6.135, 6.135], [6.135, 0.0, 6.135], [6.135, 6.135, 0.0]], tags=...)


In [8]:
parser = argparse.ArgumentParser(description='Crystal gated neural networks')
parser.add_argument('modelpath', help='path to the trained model.')
parser.add_argument('cifpath', help='path to the directory of CIF files.')
parser.add_argument('-b', '--batch-size', default=1, type=int,
                    metavar='N', help='mini-batch size (default: 100)')
parser.add_argument('-j', '--workers', default=0, type=int, metavar='N',
                    help='number of data loading workers (default: 0)')
parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--print-freq', '-p', default=10, type=int,
                    metavar='N', help='print frequency (default: 10)')

args = parser.parse_args(['model_final_submission.pth.tar','batch_template_1'])

if os.path.isfile(args.modelpath):
    print("=> loading model params '{}'".format(args.modelpath))
    model_checkpoint = torch.load(args.modelpath,
                                  map_location=lambda storage, loc: storage)
    model_args = argparse.Namespace(**model_checkpoint['args'])
    print("=> loaded model params '{}'".format(args.modelpath))
else:
    print("=> no model params found at '{}'".format(args.modelpath))

args.cuda = not args.disable_cuda and torch.cuda.is_available()


=> loading model params 'model_final_submission.pth.tar'
=> loaded model params 'model_final_submission.pth.tar'


In [9]:
dataset = CIFData('batch_template_1',radius = 12,max_num_nbr = 24)
collate_fn = collate_pool
test_loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True,
                         num_workers=args.workers, collate_fn=collate_fn,
                         pin_memory=args.cuda)

structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]
energy_model = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len,
                            atom_fea_len=model_args.atom_fea_len,
                            n_conv=model_args.n_conv,
                            h_fea_len=model_args.h_fea_len,
                            n_h=model_args.n_h,
                            classification=True if model_args.task ==
                            'classification' else False)
if args.cuda:
    energy_model.cuda()


/home/james/miniconda3/envs/james_mlmat/lib/python3.7/site-packages/pymatgen/io/cif.py:1123: UserWarning: Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


In [10]:
energy_model.eval()

for i, (input, target, batch_cif_ids) in enumerate(test_loader):
    with torch.no_grad():
        if args.cuda:
            input_var = (Variable(input[0].cuda(non_blocking=True)),
                         Variable(input[1].cuda(non_blocking=True)),
                         input[2].cuda(non_blocking=True),
                         [crys_idx.cuda(non_blocking=True) for crys_idx in input[3]])

In [11]:
import copy
print(input_var[0].shape)
ag_key = copy.deepcopy(input_var[0][0])
ag_key = ag_key.view(1,92)
ag_key_large = ag_key.expand(27,-1)
pd_key = copy.deepcopy(input_var[0][15])
pd_key = pd_key.view(1,92)
pd_key_large = pd_key.expand(27,-1)
print(ag_key_large[0,:])
print(pd_key_large[0,:])

print(input_var[1].shape)
#print(input_var[1][0])
#print(input_var[1][12499])

#set_one = copy.deepcopy(input_var[1][0])
print(ag_key_large.shape)
        

torch.Size([27, 92])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.], device='cuda:1')
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0.], device='cuda:1')
torch.Size([27, 24, 61])
torch.Size([27, 92])


In [12]:
class Normalizer(object):
    """Normalize a Tensor and restore it later. """
    def __init__(self, tensor):
        """tensor is taken as a sample to calculate the mean and std"""
        self.mean = torch.mean(tensor)
        self.std = torch.std(tensor)

    def norm(self, tensor):
        return (tensor - self.mean) / self.std

    def denorm(self, normed_tensor):
        return normed_tensor * self.std + self.mean

    def state_dict(self):
        return {'mean': self.mean,
                'std': self.std}

    def load_state_dict(self, state_dict):
        self.mean = state_dict['mean']
        self.std = state_dict['std']

In [13]:
print(args.modelpath)
normalizer = Normalizer(torch.zeros(3))
if os.path.isfile(args.modelpath):
    print("=> loading model '{}'".format(args.modelpath))
    checkpoint = torch.load(args.modelpath,
                            map_location=lambda storage, loc: storage)
    energy_model.load_state_dict(checkpoint['state_dict'])
    normalizer.load_state_dict(checkpoint['normalizer'])
    print("=> loaded model '{}' (epoch {}, validation {})"
          .format(args.modelpath, checkpoint['epoch'],
          checkpoint['best_mae_error']))
 

model_final_submission.pth.tar
=> loading model 'model_final_submission.pth.tar'
=> loaded model 'model_final_submission.pth.tar' (epoch 176, validation 1.5474278926849365)


In [14]:
def get_graph_energies(lattices):
    
 
    lattices.view(1,-1)
    batch_size = lattices.shape[0]
    lattices = lattices.view(-1)
    size = lattices.shape[0]
    ag_sites = torch.abs(((lattices + 1)/2).view(-1,1).expand(size,92))
    pd_sites = (torch.abs(1 - lattices)/2).view(-1,1).expand(size,92)
    
    ids = torch.abs(ag_sites * ag_key_large + pd_sites * pd_key_large)
    inp = (Variable(ids.cuda(non_blocking=True)),input_var[1],input_var[2],input_var[3])
    energies = energy_model(*inp)
    energies = normalizer.denorm(energies.data)
    
    return (27*energies).view(1)

In [15]:
lattices = torch.tensor([1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,-1.0]).cuda()

In [16]:
print(get_graph_energies(lattices.view(-1,27)))

tensor([-1.2444], device='cuda:1')


In [17]:
class Monte_Carlo_Simulation():
    
    def __init__(self,dim,temps,fields,J=1,samples_per_temp=1000,Eq_steps = 100):
        self.Dim = dim
        self.temps = temps
        self.fields = fields
        self.J = J
        self.samples_per_temp = samples_per_temp
        self.Eq_steps = Eq_steps
        self.Autocorr_steps = 27
        
    def get_conc(self,lattice):
        conc = torch.sum(lattice)
        return conc
    
    
    def get_energy(self,lattice,fd,window=None,window_width = None):
        int_energy = get_graph_energies(lattice)
        conc = self.get_conc(lattice)
        #print(fd)
        return int_energy - fd*conc
    
    def MC_step(self,lattice,curr_energy,temp,fd,window = None, window_width = None):

        x_pos = random.randint(0,26)
        e_init = curr_energy
        cand_lattice = torch.clone(lattice)
        if cand_lattice[x_pos] == 1.0:
            cand_lattice[x_pos] = -1.0
        else:
            cand_lattice[x_pos] = 1.0
        
        e_new = self.get_energy(cand_lattice,fd)
        delta_e = e_new - e_init
        
        if delta_e < 0:
            accepted_lattice = cand_lattice
            curr_energy = curr_energy + delta_e
        else:
            prob = -(delta_e)/(k_b*temp)
            samp = np.log(random.random())
            if samp < prob:
                accepted_lattice = cand_lattice
                curr_energy = curr_energy + delta_e
            else:
                #print('here')
                accepted_lattice = lattice
        return accepted_lattice,curr_energy
    
    def initiate_lattice(self,window = None, window_width = None):
        lattice = torch.zeros(27).cuda()
        for i in range(27):
            samp = random.random()
            if samp < 0.5:
                lattice[i] = 1.0
            else:
                lattice[i] = -1.0
        return lattice
    
    def equilibrate(self,lattice,temp,fd,window = None,window_width = None):
        for eq_time in range(self.Eq_steps):
            lattice = self.get_lattice_sample(lattice,temp,fd,window,window_width)
        return lattice
    
    def get_lattice_sample(self,lattice,temp,fd,window=None,window_width=None):
        curr_energy = self.get_energy(lattice,fd,window,window_width)
        for candidate in range(self.Autocorr_steps):
            lattice,curr_energy = self.MC_step(lattice,curr_energy,temp,fd,window,window_width)
        return lattice
    
    def run(self,start=None):
        
        energies = torch.zeros(len(self.temps),len(self.fields),self.samples_per_temp)
        concs = torch.zeros(len(self.temps),len(self.fields),self.samples_per_temp)
        for counter_t,temp in enumerate(self.temps):
            if start is not None:
                lattice = self.initiate_lattice(window=start,window_width=0.05)
            else:
                lattice = self.initiate_lattice()
                
            for counter_f,fd in enumerate(self.fields):
                lattice = self.equilibrate(lattice,temp,fd)
                #print('here')
                for sample in range(self.samples_per_temp):
                    lattice = self.get_lattice_sample(lattice,temp,fd)
                    energies[counter_t,counter_f,sample] = self.get_energy(lattice,fd)
                    concs[counter_t,counter_f,sample] = self.get_conc(lattice)
                print("done")
        return energies,concs

In [18]:
fields = np.linspace(-0.2,0.2,41)
temps = np.linspace(250,750,2)
print(fields)
print(temps)

[-0.2  -0.19 -0.18 -0.17 -0.16 -0.15 -0.14 -0.13 -0.12 -0.11 -0.1  -0.09
 -0.08 -0.07 -0.06 -0.05 -0.04 -0.03 -0.02 -0.01  0.    0.01  0.02  0.03
  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12  0.13  0.14  0.15
  0.16  0.17  0.18  0.19  0.2 ]
[250. 750.]


In [ ]:
Run_np = Monte_Carlo_Simulation(3,temps,fields)
energies_np,concs_np = Run_np.run()

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [ ]:
Run_pn = Monte_Carlo_Simulation(3,temps,np.flip(fields))
energies_pn,concs_pn = Run_pn.run()

In [21]:
print(concs_np.shape)
concs_to_store = concs_np.mean(dim=2)
concs_to_store = concs_to_store.cpu().detach().numpy()
concs_to_store = (concs_to_store+27.0)/54
print(concs_to_store.shape)

torch.Size([2, 41, 1000])
(2, 41)


In [22]:
np.savetxt("AgPd_CGC_np_Final_Submission.csv", concs_to_store, delimiter=",")

In [23]:
print(concs_to_store)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.11120717e-04
  4.81464231e-04 6.29636983e-04 1.51853205e-03 4.33925897e-01
  5.24851859e-01 5.45518517e-01 5.67925930e-01 5.86777806e-01
  6.08407438e-01 6.30074084e-01 6.48740709e-01 6.69963002e-01
  6.88888907e-01 7.07333326e-01 7.26592600e-01 7.49407470e-01
  7.74555564e-01 8.20962965e-01 8.74111116e-01 9.01740730e-01
  9.19777751e-01 9.31222200e-01 9.41851854e-01 9.51370358e-01
  9.58370328e-01 9.61888909e-01 9.67555583e-01 9.73222256e-01
  9.81518507e-01]
 [1.18516991e-03 1.99999637e-03 3.37035558e-03 4.18518204e-03
  4.96295653e-03 9.03705321e-03 1.21110985e-02 1.74074173e-02
  2.45555602e-02 4.16296162e-02 9.36666802e-02 2.01296315e-01
  3.44888896e-01 4.18925911e-01 4.61185187e-01 4.89629626e-01
  5.15333295e-01 5.37814796e-01 5.51148176e-01 5.70592582e-01
  5.88148177e-01 6.09777808e-01 6.25666678e-01 6.417

embedding.weight tensor([[ 0.0206,  0.0667,  0.0351,  ...,  0.0686,  0.0299,  0.0853],
        [ 0.0960, -0.0473, -0.0692,  ..., -0.0113,  0.0306, -0.1003],
        [-0.0171,  0.0211, -0.0760,  ...,  0.0838,  0.0092,  0.0719],
        ...,
        [ 0.0611,  0.0590, -0.0727,  ...,  0.0855,  0.0331,  0.1006],
        [ 0.0524, -0.0934,  0.0723,  ..., -0.0475, -0.0344, -0.0513],
        [-0.0598, -0.0845, -0.0519,  ..., -0.0110,  0.0386, -0.0011]],
       device='cuda:1')
embedding.bias tensor([ 0.0510, -0.1298, -0.2830, -0.3047, -0.2281, -0.1914, -0.0265, -0.3587,
        -0.2588, -0.1392, -0.0625, -0.1438, -0.1042, -0.1961, -0.1893, -0.0639,
        -0.2246, -0.2438, -0.1755, -0.0865,  0.0443, -0.2898,  0.0570, -0.4153,
        -0.0414, -0.0604, -0.0384, -0.0430,  0.0232, -0.0617, -0.0213, -0.1252,
        -0.2223, -0.2392, -0.0725,  0.0204,  0.0859, -0.1545, -0.1919, -0.1027,
        -0.1870, -0.0467, -0.1747, -0.0955, -0.1922, -0.1781, -0.0397, -0.3516,
        -0.0628, -0.1423, -0.0